<a href="https://colab.research.google.com/github/eunShim/BigDataCapstone/blob/main/bigdatacapstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================
# ✅ EdNet-KT4.zip 10,000명 샘플 분석 (7z 기반 + 재귀 탐색)
# ============================================
import os
import random
import pandas as pd
import numpy as np
from tqdm import tqdm

# --------------------------------------------
# 1. 7z 설치 및 내부 파일 목록 확인
# --------------------------------------------
# (Colab 환경일 경우 실행됨. 일반 환경에서는 주석 처리 권장)
os.system("sudo apt-get install -y p7zip-full > /dev/null")

zip_path = "/content/EdNet-KT4.zip"
list_output = os.popen(
    f"7z l '{zip_path}' | grep '.csv' | awk '{{print $6}}'"
).read().splitlines()

if not list_output:
    raise RuntimeError("❌ CSV 파일 목록을 찾을 수 없습니다. 파일이 손상되었거나 ZIP 포맷이 아닐 수 있습니다.")

total_files = len(list_output)
print(f"✅ 내부 CSV 파일 {total_files}개 탐색됨")

# --------------------------------------------
# 2. 무작위 10,000명 샘플링 후 추출
# --------------------------------------------
sample_n = min(10000, total_files)  # 전체 파일이 1만개 미만일 경우 자동 조정
sample_files = random.sample(list_output, sample_n)

output_folder = "/content/KT4_sample"
os.makedirs(output_folder, exist_ok=True)

print(f"📦 무작위 {sample_n}명 추출 중...")

for f in tqdm(sample_files, desc="📂 압축 해제 중"):
    os.system(f"7z x '{zip_path}' '{f}' -o'{output_folder}' -y > /dev/null")

print(f"✅ {sample_n}명 샘플 추출 완료!")

# --------------------------------------------
# 3. 하위 폴더까지 재귀적으로 CSV 탐색
# --------------------------------------------
csv_list = []
for root, dirs, files in os.walk(output_folder):
    for f in files:
        if f.endswith(".csv"):
            csv_list.append(os.path.join(root, f))

print(f"📁 실제 CSV 파일 개수: {len(csv_list)}")

# --------------------------------------------
# 4. 사용자별 행동 통계 계산 (7개 변수)
# --------------------------------------------
results = []
print(f"📊 {len(csv_list)}명 데이터 분석 시작...")

for i, file_path in enumerate(tqdm(csv_list, desc="📈 사용자 분석 중")):
    try:
        df = pd.read_csv(file_path)
    except Exception as e:
        print(f"⚠️ {file_path} 스킵됨 ({e})")
        continue

    user_id = os.path.splitext(os.path.basename(file_path))[0]
    df.columns = [c.strip().lower() for c in df.columns]

    if 'item_id' not in df.columns or 'action_type' not in df.columns or 'platform' not in df.columns:
        continue

    df = df[df['item_id'].notna()]
    df_valid = df[~df['item_id'].astype(str).str.startswith(('l', 'e'))]

    df_valid['problem_id'] = df_valid['item_id'].where(df_valid['item_id'].str.startswith('q'))
    df_valid['bundle_id'] = df_valid['item_id'].where(df_valid['item_id'].str.startswith('b'))
    df_valid['user_id'] = user_id

    base = (
        df_valid.groupby('user_id')
        .agg(
            n_attempts=('action_type', lambda x: (x == 'respond').sum()),
            n_submit=('action_type', lambda x: (x == 'submit').sum()),
            n_quit=('action_type', lambda x: (x == 'quit').sum()),
            n_questions=('problem_id', lambda x: x.nunique()),
            n_bundles=('bundle_id', lambda x: x.nunique())
        )
        .reset_index()
    )

    plat = (
        df_valid.groupby('user_id')
        .agg(
            platform_ratio_web=('platform', lambda x: np.mean(x == 'web')),
            platform_ratio_mobile=('platform', lambda x: np.mean(x == 'mobile'))
        )
        .reset_index()
    )

    merged = base.merge(plat, on='user_id', how='left')
    results.append(merged)

    if (i + 1) % 500 == 0:
        print(f"Processed {i+1}/{len(csv_list)} users")

# --------------------------------------------
# 5. 결과 저장
# --------------------------------------------
if len(results) > 0:
    final = pd.concat(results, ignore_index=True)
    output_path = "/content/kt4_sample_metrics_10000_fixed.csv"
    final.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"✅ 분석 완료 → {output_path} ({len(final)}명)")
else:
    print("❌ 처리할 유효한 CSV가 없습니다.")
